In [ ]:
#en este notebook hare un modelo predictivo con xgboost, bow 500 features, y luego bow c/feature hashing

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import xgboost as xgb
import re
import nltk
import string

import unidecode

import warnings
warnings.filterwarnings('ignore')

stopwords = nltk.corpus.stopwords.words('english')

C:\Anaconda\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
tweets = pd.read_csv("../data/train.csv")

In [4]:
def remove_punctuation(word):        
    clean_word = ''.join([char for char in word if char not in string.punctuation])
    return clean_word

In [5]:
def cleaning_text(text):
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    text_tokenize = tokenizer.tokenize(text)
    wordlist = []
    for word in text_tokenize:
        word = word.lower()
        word = re.sub('(?P<url>https?://[^\s]+)', ' ', word)
        word = remove_punctuation(word)
        word = re.sub(r'[^\w]', ' ', word)
        word = unidecode.unidecode(word)
        word = re.sub(r'[0-9]','', word)
        if((word != '')&(word != ' ')&(word not in stopwords)):
            wordlist.append(word)
    clean_text = ' '.join(wordlist)
    return clean_text

In [6]:
tweets["clean_text"] = tweets["text"].apply(lambda x: cleaning_text(x))
tweets.head()

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake may allah forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents asked shelter place notified officer...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders cal...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfires pou...


In [7]:
tweets.tail()

,id,keyword,location,text,target,clean_text
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant cranes holding bridge collapse nearb...
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,control wild fires california even northern pa...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,utc km volcano hawaii
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,police investigating ebike collided car little...
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1,latest homes razed northern california wildfir...


In [8]:
vectorizer = CountVectorizer()
df_text = tweets["clean_text"]
X = vectorizer.fit_transform(df_text)
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [9]:
feature_words = vectorizer.get_feature_names()
len(feature_words)

14350

In [10]:
df = pd.DataFrame(X.toarray(), columns=feature_words)
df.head()

,aa,aaa,aaaand,aaalll,aaarrrgghhh,aaemiddleaged,aal,aan,aannnd,aar,...,zones,zonewolf,zoom,zotar,zouma,zrnf,zss,zumiez,zurich,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df_filter = df.loc[:,(df.sum()>25)]
df_filter.shape

(7613, 569)

In [14]:
df_filter["target_label"] = tweets.target
X = df_filter.drop(["target_label"], axis=1)
y = df_filter["target_label"]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [16]:
model_xgb = xgb.XGBClassifier(n_estimators=50, objective='binary:logistic', max_depth=15, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.7, n_jobs=1)
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.1, max_delta_step=0, max_depth=15,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=50, n_jobs=1, num_parallel_tree=1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       validate_parameters=1, verbosity=None)

In [17]:
y_test_hat = model_xgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.768908


#### Ajustando hiper-parametros (569 features):
    - n_estimators=50, max_depth=15, learning_rate=0.1, subsample=1, colsample_bytree=0.7
    SCORE 0.768908
    - n_estimators=100, max_depth=15, learning_rate=0.1, subsample=1, colsample_bytree=0.7
    SCORE 0.776786
    - n_estimators=100, max_depth=13, learning_rate=0.1, subsample=1, colsample_bytree=0.5
    SCORE 0.772584
    - n_estimators=200, max_depth=11, learning_rate=0.1, subsample=1, colsample_bytree=0.5
    SCORE 0.787815
    - n_estimators=200, max_depth=13, learning_rate=0.1, subsample=1, colsample_bytree=0.5
    SCORE 0.784664
    - n_estimators=250, max_depth=11, learning_rate=0.1, subsample=1, colsample_bytree=0.5
    SCORE 0.789391
    - n_estimators=250, max_depth=11, learning_rate=0.1, subsample=1, colsample_bytree=0.7
    SCORE 0.790441
    - n_estimators=250, max_depth=13, learning_rate=0.1, subsample=1, colsample_bytree=0.7
    SCORE 0.796218
    - n_estimators=250, max_depth=15, learning_rate=0.1, subsample=1, colsample_bytree=0.7
    SCORE 0.796744
    - n_estimators=300, max_depth=11, learning_rate=0.1, subsample=1, colsample_bytree=0.5
    SCORE 0.793592
    - n_estimators=300, max_depth=15, learning_rate=0.1, subsample=1, colsample_bytree=0.5
    SCORE 0.798319

In [44]:
model_xgb = xgb.XGBClassifier(n_estimators=300, objective='binary:logistic', max_depth=13, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.7, n_jobs=1)
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.1, max_delta_step=0, max_depth=13,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=300, n_jobs=1, num_parallel_tree=1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=0.1, scale_pos_weight=1, subsample=1,
       tree_method='exact', validate_parameters=1, verbosity=None)

In [45]:
y_test_hat = model_xgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.796744


In [46]:
model_xgb = xgb.XGBClassifier(n_estimators=300, objective='binary:logistic', max_depth=15, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.7, n_jobs=1)

In [47]:
kfold = KFold(n_splits=4, random_state=100)
resultados = cross_val_score(model_xgb, X_train, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()*100))

Accuracy: 78.034514
